# 设置配置项适应运行需求


## 绑定运行时参数

In [2]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOACL_API_KEY")
api_base = os.getenv("LOACL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=api_base, temperature=0.3)

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages(
    [
       (
           "system",
           "用代数符号写出下面的方程，然后求解。使用格式\n\n方程：...\n解决方案：...\n\n"
       ) ,
       ("user", "{equation_statement}")
    ]
)

chain = {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()

chain.invoke({"equation_statement": "x的3次方加7等于34"})

'**方程：**\n\n\\[ x^3 + 7 = 34 \\]\n\n**解决方案：**\n\n1. 首先，将方程简化。将常数项7移到方程的右边：\n\n\\[ x^3 = 34 - 7 \\]\n\n2. 计算右边的值：\n\n\\[ x^3 = 27 \\]\n\n3. 然后，求解 \\( x \\)，即求 \\( x \\) 的立方根：\n\n\\[ x = \\sqrt[3]{27} \\]\n\n4. 计算立方根：\n\n\\[ x = 3 \\]\n\n**答案：**\n\n\\[ x = 3 \\]'

## 在运行时配置链的内部结构

通常，您可能想要尝试多种不同的做事方式，甚至向最终用户展示多种不同的做事方式。为了使这种体验尽可能简单，我们定义了两种方法。

首先是 configurable_fields 方法。这允许您配置可运行的特定字段。

其次，一个 configurable_alternatives 方法。使用此方法，您可以列出可以在运行时设置的任何特定可运行对象的替代方案。

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField


model = model.configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        description="The temperature of the LLM",
        name="LLM Temperature",
    )
)

model.configurable_fields

<bound method RunnableConfigurableFields.configurable_fields of RunnableConfigurableFields(default=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc86286a710>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc862890850>, root_client=<openai.OpenAI object at 0x7fc890222ec0>, root_async_client=<openai.AsyncOpenAI object at 0x7fc86286a770>, temperature=0.3, model_kwargs={}, openai_api_base='https://ai-yyds.com/v1'), fields={'temperature': ConfigurableField(id='llm_temperature', name='LLM Temperature', description='The temperature of the LLM', annotation=None, is_shared=False)})>

In [8]:
model.invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='74', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b80d8de-e858-418a-90f6-fb8be1b616c0-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
model.invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='37', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 11, 'total_tokens': 12, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e855345b-d31c-4e9b-a25c-91b7fd3023f3-0', usage_metadata={'input_tokens': 11, 'output_tokens': 1, 'total_tokens': 12, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
model.with_config(config={"llm_temperature": 0.9}).invoke("生成一个随机的整数，不要回答其他任何内容")

AIMessage(content='36', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 25, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c47d91f3-e303-4881-8854-144dcfff743e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 1, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## `@chain` decorator

使用“@chain”装饰器创建一个可运行程序,您还可以通过添加 @chain 装饰器将任意函数变成链。这在功能上相当于包装在 RunnableLambda 中。

通过正确跟踪您的链，这将具有提高可观察性的好处。对该函数内的可运行对象的任何调用都将被跟踪为嵌套子函数。

它还允许您像任何其他可运行程序一样使用它，将其组合成链等。

In [12]:
from langchain_core.runnables import chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt1 = ChatPromptTemplate.from_template("给我讲一个关于{topic}的故事")
prompt2 = ChatPromptTemplate.from_template("{store}\n\n对上面的故事进行修改，让故事变得更加口语化和幽默有趣。")

In [13]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOACL_API_KEY")
api_base = os.getenv("LOACL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=api_base, temperature=0.8)

In [14]:
@chain
def custom_chain1(text):
    prompt_val1=prompt1.invoke({"topic": text})
    output1 = model.invoke(prompt_val1)
    parsed_output1 = StrOutputParser().invoke(output1)
    
    chain2 = prompt2 | model | StrOutputParser()
    return chain2.invoke(parsed_output1)


@chain
def custom_chain2(text):
    chain1 = prompt1 | model | StrOutputParser()
    parsed_output1 = chain1.invoke({"topic": text})
    
    chain2 = prompt2 | model | StrOutputParser()
    return chain2.invoke(parsed_output1)


@chain
def custom_chain3(text):
    chain = prompt1 | model | StrOutputParser() | {"store": RunnablePassthrough()} | prompt2 | model | StrOutputParser()
    return chain.invoke({"topic": text})


In [16]:
custom_chain1.invoke("编程")

'### 《修复代码的艺术：一位年轻程序员的成长故事》\n\n这是一个关于一个刚从大学毕业的程序员李凯，如何从“迷茫小白”变成“老司机”的故事。李凯对编程充满了热情，带着满腔的理想主义进入了一家软件公司，准备大展拳脚。但现实很快就给了他一个“下马威”。这不仅是他个人的成长故事，也是编程世界里那些“坑”和“摔倒”的故事。\n\n#### 初入职场\n\n李凯刚进公司，接手的第一个项目就是一个超级庞大的管理系统。这个系统看起来虽然很成熟，但其实里面藏着不少“雷”，尤其是历史遗留的代码。上一个开发团队把它搞得像拼图一样，各个模块错综复杂，互相依赖，函数和类的命名就像是他们闭着眼睛随便写的——有时候连代码注释都不再是解释代码做了什么，而是解释“为什么这么做”，简直像在写小说。\n\n李凯的任务是接手这个系统进行维护，并做一些新功能的开发。起初，他满怀信心，觉得自己刚学了那么多编程理论，面对这些代码，简直就是“老司机上路，超车不在话下”。但是，当他一开始动手时，问题就来了——每当他试图理解一段代码时，都像是在解一道魔方，哪里都不对；而每次他修改了一点东西，系统就像中邪了一样，整个崩溃，回归测试后到处出问题。每次他都觉得自己像被程序给“套路”了，仿佛它是有意识地在跟他作对。\n\n#### 崩溃与困惑\n\n有一次，李凯被指派去修复一个关于用户权限管理的 bug，这个 bug 拿出来就是一颗定时炸弹。问题涉及到多个模块的交互，还有数据库层面的东西。李凯按照自己的理解，修改了代码，然后提交。结果测试人员来了一句：“权限管理还不行，而且有时候程序崩溃了。”李凯心想：“完了，崩了就崩了呗，给我点时间我再修一修。”于是他再次修改，增加了更多的日志和调试信息，好像这些日志就能帮他找出bug的真身。\n\n但是，问题还是没解决，李凯开始怀疑自己是不是根本不适合做程序员了。每当遇到难题，他就只能靠直觉和一点点“灵光一闪”的思路去解谜，而不是系统地分析问题。那种无力感，就像一只脚踩在水里，怎么都没法站稳。\n\n有一天，他和资深开发者王工讨论问题，王工看了一眼李凯的代码，笑了笑说：“其实，编程最重要的，不是瞎琢磨解决问题，而是理解问题的本质。就像侦探一样，问题就藏在细节里，等着你去发现。”\n\n#### 转折：从错误中学习\n\n这句话像一针强心剂，李凯渐渐从迷茫中恢复过来，决定改变自己的编

In [17]:
custom_chain2.invoke("编程")

'### 《一行代码改变了世界》\n\n这是一个关于编程的故事，讲的是一个程序员通过一行代码的灵感，如何改变了整个行业的命运。故事发生在2000年代初期的硅谷，主人公叫艾伦，是个年轻的程序员，正面临着人生的一个关键时刻。\n\n#### 初入编程的艾伦\n\n艾伦从小就对计算机有种“莫名的吸引力”。家里没什么钱，但他有个宝贵的资源——一台老旧的父亲的电脑。那时候，他还是个高中生。艾伦的第一个编程项目是写个“猜数字”游戏，虽然那代码简直像拼图一样简单，但当他看到电脑“思考”并给出答案时，他内心的小火花瞬间燃烧起来——“哇，原来我也能教电脑做事！”。从那一刻起，他决定走上编程这条“不归路”。\n\n大学毕业后，艾伦加入了一家初创公司，开始了正式的程序员生涯。公司主打的产品是一个云存储系统，听起来很炫，但实际操作起来，技术问题层出不穷。每当客户抱怨数据慢得像乌龟一样时，艾伦和他的团队只能低头默默修复代码，心里都在想：这公司真的能行吗？\n\n#### 项目的困境\n\n在这个项目中，艾伦和其他工程师们一直在努力，然而，面对一个难解的技术难题——**高并发下，数据存取速度超级慢**，整个团队感觉就像被困在了“性能地狱”。无论怎么努力优化代码，似乎都看不到希望的曙光。客户反馈一片差评，很多用户开始放弃使用这个平台，公司一度快要挂掉了。艾伦心里压力山大，觉得自己随时可能被炒鱿鱼。\n\n有一天晚上，艾伦又加班加点到很晚，桌子上堆满了纸张和代码审查记录。他已经累得眼皮打架，突然起身去倒了杯咖啡，想清清脑袋。手拿咖啡杯，他无意识地盯着代码编辑器屏幕，眼神空洞地游移……突然，他的视线停在了一个不起眼的小细节上：**缓存机制**。那个小小的模块，虽然理论上应该能提升性能，但之前的设计问题让它根本没能发挥作用。\n\n#### 灵感的瞬间\n\n那一瞬间，艾伦突然想到了：**如果把缓存刷新策略改成基于数据变化来实时失效，而不是固定时间间隔刷新呢？** 噢，天哪！这么简单的改动居然没想到？他心里一阵激动，像是抓住了“魔法钥匙”。他迅速修改了一行代码，把缓存刷新机制改了过来。测试一下，哇塞，结果让他差点掉了咖啡杯：系统响应速度居然提高了80%！艾伦简直要疯了，他赶紧把这个改动提交给团队。\n\n#### 影响与突破\n\n团队经过进一步测试，确认这行简单的代码确实解决了性能瓶颈。随后，他们把

In [18]:
custom_chain3.invoke("编程")

'好的，下面是修改后的版本，故事更加口语化，带有一些幽默和趣味性：\n\n---\n\n### 故事的开端\n\n在一个安静的小镇上，有个年轻女孩，名字叫李华。李华从小就喜欢电脑，玩游戏、画画、看网站，她几乎对电脑的所有功能都充满了好奇心。可是呢，她完全没想过自己有一天会变成一个程序员，直到高中的那门**计算机科学**课。\n\n那时的李华，编程对她来说简直像是天书。她听到别人说“代码”时，总是想，“是不是很难的东西？我能搞定吗？”直到有一天，她遇到了她的第一个编程语言——**Python**。她才发现，原来编程竟然可以这么有趣！不是枯燥的数字和符号，而是能让计算机听话的魔法。\n\n### 第一次编程的尝试\n\n李华的第一段代码简直简单到不行，连她自己都忍不住笑了出来。就是那种经典的“Hello, World!”程序。她按下了“运行”，屏幕上跳出来那句“Hello, World!”时，李华差点高兴得要从椅子上跳起来。她甚至盯着那行字看了好久，心想：“天呐，这居然是我写的！这电脑真的在听我说话！”\n\n那一刻，李华决定，编程不再是个冷冰冰的东西，而是个能让她随心所欲地创造的工具。从那以后，她开始疯狂学习编程，不停地试着写小程序——比如计算器、猜数字游戏之类的。每当一个程序运行成功，她就会觉得自己像个小小的魔法师，自己也能让电脑做一些神奇的事。\n\n### 迎接挑战\n\n当然，李华的编程之路并非一帆风顺。随着她的技能逐渐提升，挑战也越来越大。有一次，她决定自己做个小网页应用，想着用HTML和CSS设计个漂亮页面，再用JavaScript加上一点互动，应该没啥问题。可是当她开始想要存储数据和做后端功能时，问题来了。\n\n她一开始尝试把数据存到本地文件里，结果没几天就发现，这根本不行——数据根本存不下去，问题一大堆。于是，她转战**MySQL**，想着，“数据库不就是存数据吗，应该很简单吧？”结果一打开数据库，她就傻眼了：表格设计、数据关系、查询语法，全都像是一团乱麻。每次尝试查询时，数据库好像都在给她开玩笑，什么“连接失败”，“语法错误”，甚至“空结果”……那一刻，她差点要放弃了。\n\n但在一次和老师的交流后，她决定坚持下去。李华开始疯狂刷教程，看了无数个视频，跟着手一键键盘打了一堆SQL，终于在几个月后，成功做出了一个能够保存用户信息并实现注册登录的网页应